In [4]:
import pandas                   as pd
from tkinter import *
from tkinter.constants import DISABLED, NORMAL, TOP, BOTH
from tkinter.ttk import Button, Label
import joblib
import webbrowser
from mpl_toolkits.basemap import Basemap
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib import pyplot   as plt
import math
from tkinter import ttk
import numpy
from tkinter import messagebox

PQ_viz = pd.read_csv("bronze.csv", sep = ",")
PQ_viz['time'] = PQ_viz['time'].str[:19]
PQ_viz['time'] = pd.to_datetime(PQ_viz['time'])
PQ_viz = PQ_viz[["time", "latitude", "longitude", "depth", "mag", "nst", "gap", "dmin", "rms", "net", "id", "updated"]]
model_KNN = joblib.load('KNN-Predictquake.sav')

window = Tk()
window.title("PredictQuake - What is the strength of a potential earthquake in your region (coordinate)? ")
window.geometry('800x800')
window.iconbitmap("PredictQuake.ico")
window.resizable(0,0)


canvas=Canvas(window, height=800, width=800)
bg = PhotoImage(file = "bg.png", master=window)
canvas.pack()
canvas.create_image( 0, 0, image = bg, anchor = "nw")
canvas.create_line(250, 10, 250, 800, dash=(4, 2), fill='dark gray')
canvas.create_line(10, 170, 250, 170, dash=(4, 2), fill='dark gray')
canvas.create_line(10, 410, 800, 410, dash=(4, 2), fill='dark gray')
Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.46, anchor='center')
 

frame = Frame(canvas, width=500, height=360, highlightbackground="black", highlightthickness=1, bg = "white")
frame.place(relx = 0.345, rely = 0.04)
frame.pack_propagate(False)

bottomframe = Frame(canvas, width=500, height=317, highlightbackground="black", highlightthickness=1, bg = "white")
bottomframe.place(relx = 0.345, rely = 0.56)
bottomframe.pack_propagate(False)

# Your information
Label(window, text="Your information", background='#dbeff9', font=("Segoe UI", 10, "bold")).place(relx=0.15, rely=0.02, anchor='center')

lbl = Label(window, text="Do you not know your coordinates?", font=("Segoe UI", 8), background='#dbeff9')
lbl.place(relx=0.15, rely=0.125, anchor='center')
lbllink = Label(window, text="Click here to see your coordinates.", font=("Segoe UI", 8), foreground='blue', cursor="hand2", underline=0, background='#dbeff9')
lbllink.place(relx=0.15, rely=0.145, anchor='center')
lbllink.bind("<Button-1>", lambda e: webbrowser.open_new("https://www.gps-coordinates.net/my-location"))

Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
lbl = Label(window, text="Tips for category:", background='#dbeff9')
lbl.place(relx=0.15, rely=0.58, anchor='e')

lat = StringVar()
lon = StringVar()

def model(): 
    x_test = {"latitude": float(lat.get()), "longitude": float(lon.get())}
    x_test = pd.DataFrame([x_test])
    y_pred = model_KNN.predict(x_test)
    lbl = Label(window, text=float(y_pred.round(1)), font=("Arial", 27)).place(relx=0.15, rely=0.33, anchor='center')

    for widget in frame.winfo_children():
        widget.destroy()

    for widget in bottomframe.winfo_children():
        widget.destroy()

    if(float(y_pred.round(1)) <= 2):
                Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
                Label(window, text=float(y_pred.round(1)), foreground = "gray", background = '#dbeff9', font=("Arial", 27)).place(relx=0.15, rely=0.33, anchor='center')
                Label(window, text="    LOW    ", foreground = "gray", font=("Arial", 28), background = '#dbeff9').place(relx=0.15, rely=0.46, anchor='center')
                Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
                Label(window, text="Tips for category:", background='#dbeff9').place(relx=0.15, rely=0.58, anchor='e')
                Label(window, text="LOW", foreground = "gray", font=("Arial", 13), background='#dbeff9').place(relx=0.22, rely=0.58, anchor='center')
                #Tips
                Label(window, text="There is next to no hazard in your\narea. Great news!", background='#dbeff9').place(relx=0.03, rely=0.65, anchor='w')
                Label(window, text="You could read about general\nprocedures about earthquakes by\nyour government.", background='#dbeff9').place(relx=0.03, rely=0.71, anchor='w')
                Label(window, text="Risk of damage to property or\ninjury due to earthquakes is low.", background='#dbeff9').place(relx=0.03, rely=0.78, anchor='w')
                Label(window, text="Know what the fastest way is to\nyour local earthquake shelter.", background='#dbeff9').place(relx=0.03, rely=0.84, anchor='w')
                Label(window, text="Source: Red Cross - Earthquake Preparation", font=("Segoe UI", 8, "italic"), background='#b2e7a2').place(relx=0.02, rely=0.97, anchor='w')

                fig = Figure(figsize=(5, 5))
                fig.clear()
                ax = fig.add_subplot(1,1,1)

                user_coord_latitude = float(lat.get())
                user_coord_longitude = float(lon.get())

                user_coord_longitude_min = user_coord_longitude - (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_longitude_max = user_coord_longitude + (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_latitude_min = user_coord_latitude - 0.9
                user_coord_latitude_max = user_coord_latitude + 0.9

                output_dataset = PQ_viz.loc[(PQ_viz['latitude'].between(user_coord_latitude_min, user_coord_latitude_max))]
                output_dataset = output_dataset.loc[(output_dataset['longitude'].between(user_coord_longitude_min, user_coord_longitude_max))]
                output_dataset = output_dataset[["time", "latitude", "longitude", "depth", "mag"]].sort_values(by = ["time", "mag"], ascending = False)
                output_col = ["Date", "Latitude", "Longitude", "Depth", "Magnitude"]
                output_row = output_dataset.to_numpy().tolist()

                trv = ttk.Treeview(bottomframe, selectmode = "browse")
                trv['height']=14
                trv['show']="headings"
                trv['columns']=output_col
                trv.column(["Date"], width = 137, anchor = "c")
                trv.column(["Latitude"], width = 80, anchor = "c")
                trv.column(["Longitude"], width = 80, anchor = "c")
                trv.column(["Depth"], width = 60, anchor = "c")
                trv.column(["Magnitude"], width = 85, anchor = "c")
                for i in output_col:
                        trv.heading(i, text = i)
                for dt in output_row:
                        v = [r for r in dt]
                        trv.insert("", END, iid=v[0], values=v)

                scroll = ttk.Scrollbar(bottomframe, orient="vertical", command=trv.yview)
                scroll.place(x=16, y=29, height=255)
                trv.configure(yscrollcommand=scroll.set)
                                
                m = Basemap(projection='mill',  llcrnrlat=user_coord_latitude_min,
                                                                        urcrnrlat=user_coord_latitude_max, 
                                                                        llcrnrlon=user_coord_longitude_min, 
                                                                        urcrnrlon=user_coord_longitude_max, resolution='h', ax=ax)
                longitudes = PQ_viz["longitude"].tolist()
                latitudes = PQ_viz["latitude"].tolist()
                x,y = m(longitudes,latitudes)
                xpt,ypt = m(user_coord_longitude,user_coord_latitude)
                                
                m.drawcoastlines(color = '#2296ab')
                m.fillcontinents(color='#D4D93D',lake_color='#21AFBF')
                m.drawmapboundary(fill_color='#21AFBF')
                m.drawcountries(color = '#2296ab')
                sca = m.scatter(x, y, marker = "x", s=PQ_viz["depth"].sort_values()/8, c=PQ_viz["mag"].sort_values(), cmap="seismic", alpha=0.9)
                fig.colorbar(sca, label='Magnitude')
                m.plot(xpt,ypt,'#ad3034', marker = 'v')

                Label(frame, text="Seismic magnitudes are used to describe the overall strength or intensity of an earthquake, from 1 to 10.", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.97, anchor='w')
                Label(frame, text="Source: U.S. Geological Survey - Earthquakes from 1930 to 2018", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.935, anchor='w')

                canvas2 = FigureCanvasTkAgg(fig, master=frame)
                canvas2.draw()
                canvas2.get_tk_widget().pack(padx = 20, pady = 30, anchor = "ne")
                trv.pack(side=TOP, padx = 20, pady = 30, anchor = "se")

    elif((float(y_pred.round(2)) > 2) and (float(y_pred.round(2)) < 4)):
                Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
                Label(window, text=float(y_pred.round(1)), foreground = "green", font=("Arial", 27), background='#dbeff9').place(relx=0.15, rely=0.33, anchor='center')
                Label(window, text=" NORMAL ", foreground = "green", font=("Arial", 28), background='#dbeff9').place(relx=0.15, rely=0.46, anchor='center')
                Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
                Label(window, text="Tips for category:", background='#dbeff9').place(relx=0.15, rely=0.58, anchor='e')    
                Label(window, text="NORMAL", foreground = "green", font=("Arial", 13), background='#dbeff9').place(relx=0.22, rely=0.58, anchor='center')
                #Tips
                Label(window, text="Fasten bookcases, hutches, and\nfreestanding shelving to walls.", background='#dbeff9').place(relx=0.03, rely=0.65, anchor='w')
                Label(window, text="Place large, heavy objects, as well as\nfragile objects, on lower shelves.", background='#dbeff9').place(relx=0.03, rely=0.71, anchor='w')
                Label(window, text="Know where your electricity, gas,\nand water switches and valves are\nlocated and how to shut them off.", background='#dbeff9').place(relx=0.03, rely=0.78, anchor='w')
                Label(window, text="Make a record of your personal\nproperty by taking photos/videos\nof your belongings. Store these\nrecords in a safe place.", background='#dbeff9').place(relx=0.03, rely=0.86, anchor='w')
                Label(window, text="Source: Red Cross - Earthquake Preparation", font=("Segoe UI", 8, "italic"), background='#b2e7a2').place(relx=0.02, rely=0.97, anchor='w')

                fig = Figure(figsize=(5, 5))
                fig.clear()
                ax = fig.add_subplot(1,1,1)

                user_coord_latitude = float(lat.get())
                user_coord_longitude = float(lon.get())

                user_coord_longitude_min = user_coord_longitude - (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_longitude_max = user_coord_longitude + (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_latitude_min = user_coord_latitude - 0.9
                user_coord_latitude_max = user_coord_latitude + 0.9

                output_dataset = PQ_viz.loc[(PQ_viz['latitude'].between(user_coord_latitude_min, user_coord_latitude_max))]
                output_dataset = output_dataset.loc[(output_dataset['longitude'].between(user_coord_longitude_min, user_coord_longitude_max))]
                output_dataset = output_dataset[["time", "latitude", "longitude", "depth", "mag"]].sort_values(by = ["time", "mag"], ascending = False)
                output_col = ["Date", "Latitude", "Longitude", "Depth", "Magnitude"]
                output_row = output_dataset.to_numpy().tolist()

                trv = ttk.Treeview(bottomframe, selectmode = "browse")
                trv['height']=14
                trv['show']="headings"
                trv['columns']=output_col
                trv.column(["Date"], width = 137, anchor = "c")
                trv.column(["Latitude"], width = 80, anchor = "c")
                trv.column(["Longitude"], width = 80, anchor = "c")
                trv.column(["Depth"], width = 60, anchor = "c")
                trv.column(["Magnitude"], width = 85, anchor = "c")
                for i in output_col:
                        trv.heading(i, text = i)
                for dt in output_row:
                        v = [r for r in dt]
                        trv.insert("", END, iid=v[0], values=v)
                
                scroll = ttk.Scrollbar(bottomframe, orient="vertical", command=trv.yview)
                scroll.place(x=16, y=29, height=255)
                trv.configure(yscrollcommand=scroll.set)
                                
                m = Basemap(projection='mill',  llcrnrlat=user_coord_latitude_min,
                                                                        urcrnrlat=user_coord_latitude_max, 
                                                                        llcrnrlon=user_coord_longitude_min, 
                                                                        urcrnrlon=user_coord_longitude_max, resolution='h', ax=ax)
                longitudes = PQ_viz["longitude"].tolist()
                latitudes = PQ_viz["latitude"].tolist()
                x,y = m(longitudes,latitudes)
                xpt,ypt = m(user_coord_longitude,user_coord_latitude)
                                
                m.drawcoastlines(color = '#2296ab')
                m.fillcontinents(color='#D4D93D',lake_color='#21AFBF')
                m.drawmapboundary(fill_color='#21AFBF')
                m.drawcountries(color = '#2296ab')
                sca = m.scatter(x, y, marker = "x", s=PQ_viz["depth"].sort_values()/8, c=PQ_viz["mag"].sort_values(), cmap="seismic", alpha=0.9)
                fig.colorbar(sca, label='Magnitude')
                m.plot(xpt,ypt,'#ad3034', marker = 'v')

                Label(frame, text="Seismic magnitudes are used to describe the overall strength or intensity of an earthquake, from 1 to 10.", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.97, anchor='w')
                Label(frame, text="Source: U.S. Geological Survey - Earthquakes from 1930 to 2018", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.935, anchor='w')

                canvas2 = FigureCanvasTkAgg(fig, master=frame)
                canvas2.draw()
                canvas2.get_tk_widget().pack(padx = 20, pady = 30, anchor = "ne")
                trv.pack(side=TOP, padx = 20, pady = 30, anchor = "se")
                
    elif((float(y_pred.round(2)) > 3.9) and (float(y_pred.round(2)) < 6)):
                Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
                Label(window, text=float(y_pred.round(1)), foreground = "#f2a902", font=("Arial", 27), background='#dbeff9').place(relx=0.15, rely=0.33, anchor='center')
                Label(window, text=" MEDIUM ", foreground = "#f2a902", font=("Arial", 28), background='#dbeff9').place(relx=0.15, rely=0.46, anchor='center')
                Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
                Label(window, text="Tips for category:", background='#dbeff9').place(relx=0.15, rely=0.58, anchor='e')    
                Label(window, text="MEDIUM", foreground = "#f2a902", font=("Arial", 13), background='#dbeff9').place(relx=0.22, rely=0.58, anchor='center')
                #Tips
                Label(window, text="Identify safe places in each room of\nyour home.", background='#dbeff9').place(relx=0.03, rely=0.63, anchor='w')
                Label(window, text="Practice Drop, Cover, and Hold On\nwith each member of your\nhousehold.", background='#dbeff9').place(relx=0.03, rely=0.69, anchor='w')
                Label(window, text="Keep extra cash and change.\nIf electricity is out, you will not be\nable to use an ATM", background='#dbeff9').place(relx=0.03, rely=0.76, anchor='w')
                Label(window, text="Make copies of vital records and\nkeep them in a safe deposit box in\nanother city or state. "
                                        "Make sure\nyour originals are stored safely.\nKeep copies of important\ndocuments (like ID, passport,\ninsurance cards) in your\n'Earthquake Bag'.", background='#dbeff9').place(relx=0.03, rely=0.875, anchor='w')
                Label(window, text="Source: Red Cross - Earthquake Preparation", font=("Segoe UI", 8, "italic"), background='#b2e7a2').place(relx=0.02, rely=0.97, anchor='w')

                fig = Figure(figsize=(5, 5))
                fig.clear()
                ax = fig.add_subplot(1,1,1)

                user_coord_latitude = float(lat.get())
                user_coord_longitude = float(lon.get())

                user_coord_longitude_min = user_coord_longitude - (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_longitude_max = user_coord_longitude + (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_latitude_min = user_coord_latitude - 0.9
                user_coord_latitude_max = user_coord_latitude + 0.9

                output_dataset = PQ_viz.loc[(PQ_viz['latitude'].between(user_coord_latitude_min, user_coord_latitude_max))]
                output_dataset = output_dataset.loc[(output_dataset['longitude'].between(user_coord_longitude_min, user_coord_longitude_max))]
                output_dataset = output_dataset[["time", "latitude", "longitude", "depth", "mag"]].sort_values(by = ["time", "mag"], ascending = False)
                output_col = ["Date", "Latitude", "Longitude", "Depth", "Magnitude"]
                output_row = output_dataset.to_numpy().tolist()

                trv = ttk.Treeview(bottomframe, selectmode = "browse")
                trv['height']=14
                trv['show']="headings"
                trv['columns']=output_col
                trv.column(["Date"], width = 137, anchor = "c")
                trv.column(["Latitude"], width = 80, anchor = "c")
                trv.column(["Longitude"], width = 80, anchor = "c")
                trv.column(["Depth"], width = 60, anchor = "c")
                trv.column(["Magnitude"], width = 85, anchor = "c")
                for i in output_col:
                        trv.heading(i, text = i)
                for dt in output_row:
                        v = [r for r in dt]
                        trv.insert("", END, iid=v[0], values=v)
                
                scroll = ttk.Scrollbar(bottomframe, orient="vertical", command=trv.yview)
                scroll.place(x=16, y=29, height=255)
                trv.configure(yscrollcommand=scroll.set)

                m = Basemap(projection='mill',  llcrnrlat=user_coord_latitude_min,
                                                                        urcrnrlat=user_coord_latitude_max, 
                                                                        llcrnrlon=user_coord_longitude_min, 
                                                                        urcrnrlon=user_coord_longitude_max, resolution='h', ax=ax)
                longitudes = PQ_viz["longitude"].tolist()
                latitudes = PQ_viz["latitude"].tolist()
                x,y = m(longitudes,latitudes)
                xpt,ypt = m(user_coord_longitude,user_coord_latitude)
                                
                m.drawcoastlines(color = '#2296ab')
                m.fillcontinents(color='#D4D93D',lake_color='#21AFBF')
                m.drawmapboundary(fill_color='#21AFBF')
                m.drawcountries(color = '#2296ab')
                sca = m.scatter(x, y, marker = "x", s=PQ_viz["depth"].sort_values()/8, c=PQ_viz["mag"].sort_values(), cmap="seismic", alpha=0.9)
                fig.colorbar(sca, label='Magnitude')
                m.plot(xpt,ypt,'#ad3034', marker = 'v')

                Label(frame, text="Seismic magnitudes are used to describe the overall strength or intensity of an earthquake, from 1 to 10.", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.97, anchor='w')
                Label(frame, text="Source: U.S. Geological Survey - Earthquakes from 1930 to 2018", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.935, anchor='w')

                canvas2 = FigureCanvasTkAgg(fig, master=frame)
                canvas2.draw()
                canvas2.get_tk_widget().pack(padx = 20, pady = 30, anchor = "ne")
                trv.pack(side=TOP, padx = 20, pady = 30, anchor = "se")
                

    elif((float(y_pred.round(2)) > 5.9) and (float(y_pred.round(2)) < 8)):
                Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
                Label(window, text=float(y_pred.round(1)), foreground = "#ff681d", font=("Arial", 27), background='#dbeff9').place(relx=0.15, rely=0.33, anchor='center')
                Label(window, text="  HIGH  ", foreground = "#ff681d", font=("Arial", 28), background='#dbeff9').place(relx=0.15, rely=0.46, anchor='center')
                Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
                Label(window, text="Tips for category:", background='#cccccc').place(relx=0.15, rely=0.58, anchor='e')    
                Label(window, text="HIGH", foreground = "#ff681d", font=("Arial", 13), background='#cccccc').place(relx=0.22, rely=0.58, anchor='center')
                #Tips
                Label(window, text="Adding anchor bolts or steel plates between\nyour home and its foundation.", background='#cccccc').place(relx=0.01, rely=0.65, anchor='w')
                Label(window, text="Bracing the inside of your home's cripple\nwall (the short wood-stud wall between the\ntop of the foundation wall and the first floor)\nwith sheathing.", background='#cccccc').place(relx=0.01, rely=0.72, anchor='w')
                Label(window, text="Bracing unreinforced chimneys, masonry\nand concrete walls and foundations.", background='#cccccc').place(relx=0.01, rely=0.79, anchor='w')
                Label(window, text="Become familiar with your community's\ndisaster preparedness plans and create\na family plan with escape routes,\nan emergency meeting place and a contact\n"
                        "point to communicate with concerned\nrelatives.", background='#cccccc').place(relx=0.01, rely=0.88, anchor='w')
                Label(window, text="Source: Red Cross - Earthquake Preparation", font=("Segoe UI", 8, "italic"), background='#b2e7a2').place(relx=0.02, rely=0.97, anchor='w')

                fig = Figure(figsize=(5, 5))
                fig.clear()
                ax = fig.add_subplot(1,1,1)

                user_coord_latitude = float(lat.get())
                user_coord_longitude = float(lon.get())

                user_coord_longitude_min = user_coord_longitude - (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_longitude_max = user_coord_longitude + (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_latitude_min = user_coord_latitude - 0.9
                user_coord_latitude_max = user_coord_latitude + 0.9

                output_dataset = PQ_viz.loc[(PQ_viz['latitude'].between(user_coord_latitude_min, user_coord_latitude_max))]
                output_dataset = output_dataset.loc[(output_dataset['longitude'].between(user_coord_longitude_min, user_coord_longitude_max))]
                output_dataset = output_dataset[["time", "latitude", "longitude", "depth", "mag"]].sort_values(by = ["time", "mag"], ascending = False)
                output_col = ["Date", "Latitude", "Longitude", "Depth", "Magnitude"]
                output_row = output_dataset.to_numpy().tolist()

                trv = ttk.Treeview(bottomframe, selectmode = "browse")
                trv['height']=14
                trv['show']="headings"
                trv['columns']=output_col
                trv.column(["Date"], width = 137, anchor = "c")
                trv.column(["Latitude"], width = 80, anchor = "c")
                trv.column(["Longitude"], width = 80, anchor = "c")
                trv.column(["Depth"], width = 60, anchor = "c")
                trv.column(["Magnitude"], width = 85, anchor = "c")
                for i in output_col:
                        trv.heading(i, text = i)
                for dt in output_row:
                        v = [r for r in dt]
                        trv.insert("", END, iid=v[0], values=v)

                scroll = ttk.Scrollbar(bottomframe, orient="vertical", command=trv.yview)
                scroll.place(x=16, y=29, height=255)
                trv.configure(yscrollcommand=scroll.set)
                                
                m = Basemap(projection='mill',  llcrnrlat=user_coord_latitude_min,
                                                                        urcrnrlat=user_coord_latitude_max, 
                                                                        llcrnrlon=user_coord_longitude_min, 
                                                                        urcrnrlon=user_coord_longitude_max, resolution='h', ax=ax)
                longitudes = PQ_viz["longitude"].tolist()
                latitudes = PQ_viz["latitude"].tolist()
                x,y = m(longitudes,latitudes)
                xpt,ypt = m(user_coord_longitude,user_coord_latitude)
                                
                m.drawcoastlines(color = '#2296ab')
                m.fillcontinents(color='#D4D93D',lake_color='#21AFBF')
                m.drawmapboundary(fill_color='#21AFBF')
                m.drawcountries(color = '#2296ab')
                sca = m.scatter(x, y, marker = "x", s=PQ_viz["depth"].sort_values()/8, c=PQ_viz["mag"].sort_values(), cmap="seismic", alpha=0.9)
                fig.colorbar(sca, label='Magnitude')
                m.plot(xpt,ypt,'#ad3034', marker = 'v')

                Label(frame, text="Seismic magnitudes are used to describe the overall strength or intensity of an earthquake, from 1 to 10.", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.97, anchor='w')
                Label(frame, text="Source: U.S. Geological Survey - Earthquakes from 1930 to 2018", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.935, anchor='w')

                canvas2 = FigureCanvasTkAgg(fig, master=frame)
                canvas2.draw()
                canvas2.get_tk_widget().pack(padx = 20, pady = 30, anchor = "ne")
                trv.pack(side=TOP, padx = 20, pady = 30, anchor = "se")
                
            
    elif(float(y_pred.round(2)) > 7.9):
                Frame(window, width=200, height=50, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.15, rely=0.3317, anchor='center')
                Label(window, text=float(y_pred.round(1)), foreground = "#c30000", font=("Arial", 27), background='#dbeff9').place(relx=0.15, rely=0.33, anchor='center')
                Label(window, text="EXTREME", foreground = "#c30000", font=("Arial", 28), background='#dbeff9').place(relx=0.15, rely=0.46, anchor='center')
                Frame(window, width=200, height=320, highlightbackground="black", highlightthickness=1, bg = "#dbeff9").place(relx=0.025, rely=0.76, anchor='w')
                Label(window, text="Tips for category:", background='#cccccc').place(relx=0.15, rely=0.58, anchor='e')    
                Label(window, text="EXTREME", foreground = "#c30000", font=("Arial", 13), background='#cccccc').place(relx=0.22, rely=0.58, anchor='center')
                #Tips
                Label(window, text="Make sure you have sufficient emergency supplies,\nincluding clean water, non-perishable foods, prescription drugs,\nfirst aid kit, batteries, radio, and food for pets.", background='#cccccc').place(relx=0.01, rely=0.65, anchor='w')
                Label(window, text="Keep important documents such as passports\nor identity documents with you.", background='#cccccc').place(relx=0.01, rely=0.72, anchor='w')
                Label(window, text="Learn how to act in the case of an earthquake\nby participating in simulations in your\ncommunity/workplace/school.", background='#cccccc').place(relx=0.01, rely=0.79, anchor='w')
                Label(window, text="Find out if your home is in need of earthquake\nretrofitting and eligible for a grant.", background='#cccccc').place(relx=0.01, rely=0.88, anchor='w')
                Label(window, text="Source: Red Cross - Earthquake Preparation", font=("Segoe UI", 8, "italic"), background='#b2e7a2').place(relx=0.02, rely=0.97, anchor='w')

                fig = Figure(figsize=(5, 5))
                fig.clear()
                ax = fig.add_subplot(1,1,1)

                user_coord_latitude = float(lat.get())
                user_coord_longitude = float(lon.get())

                user_coord_longitude_min = user_coord_longitude - (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_longitude_max = user_coord_longitude + (0.9 / math.cos(user_coord_latitude*math.pi/180))
                user_coord_latitude_min = user_coord_latitude - 0.9
                user_coord_latitude_max = user_coord_latitude + 0.9

                output_dataset = PQ_viz.loc[(PQ_viz['latitude'].between(user_coord_latitude_min, user_coord_latitude_max))]
                output_dataset = output_dataset.loc[(output_dataset['longitude'].between(user_coord_longitude_min, user_coord_longitude_max))]
                output_dataset = output_dataset[["time", "latitude", "longitude", "depth", "mag"]].sort_values(by = ["time", "mag"], ascending = False)
                output_col = ["Date", "Latitude", "Longitude", "Depth", "Magnitude"]
                output_row = output_dataset.to_numpy().tolist()

                trv = ttk.Treeview(bottomframe, selectmode = "browse")
                trv['height']=14
                trv['show']="headings"
                trv['columns']=output_col
                trv.column(["Date"], width = 137, anchor = "c")
                trv.column(["Latitude"], width = 80, anchor = "c")
                trv.column(["Longitude"], width = 80, anchor = "c")
                trv.column(["Depth"], width = 60, anchor = "c")
                trv.column(["Magnitude"], width = 85, anchor = "c")
                for i in output_col:
                        trv.heading(i, text = i)
                for dt in output_row:
                        v = [r for r in dt]
                        trv.insert("", END, iid=v[0], values=v)

                scroll = ttk.Scrollbar(bottomframe, orient="vertical", command=trv.yview)
                scroll.place(x=16, y=29, height=255)
                trv.configure(yscrollcommand=scroll.set)
                                
                m = Basemap(projection='mill',  llcrnrlat=user_coord_latitude_min,
                                                                        urcrnrlat=user_coord_latitude_max, 
                                                                        llcrnrlon=user_coord_longitude_min, 
                                                                        urcrnrlon=user_coord_longitude_max, resolution='h', ax=ax)
                longitudes = PQ_viz["longitude"].tolist()
                latitudes = PQ_viz["latitude"].tolist()
                x,y = m(longitudes,latitudes)
                xpt,ypt = m(user_coord_longitude,user_coord_latitude)
                                
                m.drawcoastlines(color = '#2296ab')
                m.fillcontinents(color='#D4D93D',lake_color='#21AFBF')
                m.drawmapboundary(fill_color='#21AFBF')
                m.drawcountries(color = '#2296ab')
                sca = m.scatter(x, y, marker = "x", s=PQ_viz["depth"].sort_values()/8, c=PQ_viz["mag"].sort_values(), cmap="seismic", alpha=0.9)
                fig.colorbar(sca, label='Magnitude')
                m.plot(xpt,ypt,'#ad3034', marker = 'v')

                Label(frame, text="Seismic magnitudes are used to describe the overall strength or intensity of an earthquake, from 1 to 10.", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.97, anchor='w')
                Label(frame, text="Source: U.S. Geological Survey - Earthquakes from 1930 to 2018", font=("Segoe UI", 7, "italic"), background='white').place(relx=0.02, rely=0.935, anchor='w')

                canvas2 = FigureCanvasTkAgg(fig, master=frame)
                canvas2.draw()
                canvas2.get_tk_widget().pack(padx = 20, pady = 30, anchor = "ne")
                trv.pack(side=TOP, padx = 20, pady = 30, anchor = "se")

btn2 = Button(window, text="Submit", command = model)
btn2.place(relx=0.10, rely=0.18, anchor='center')

def help():
        messagebox.showinfo(title="How to use PredictQuake?", message="Welcome to PredictQuake! This application gives you insights in the seismitivity of your area and predicts the projected magnitude. How does it work? You can start with filling in your coordinates. Do you not know your coordinates? Please check the link above this button to see your coordinates. Click on 'SUBMIT' after filling in your coordinate. After a little wait, the projected magnitude will be shown. Good luck!")

btn2 = Button(window, text="Help", command = help)
btn2.place(relx=0.20, rely=0.18, anchor='center')

lbl = Label(window, text="Longitude", background='#dbeff9')
lbl.place(relx=0.20, rely=0.07, anchor='center')

longitude = Entry(window, text = lon, width=10)
longitude.place(relx=0.20, rely=0.1, anchor='center')

lbl = Label(window, text="Latitude", background='#dbeff9')
lbl.place(relx=0.10, rely=0.07, anchor='center')

latitude = Entry(window, text = lat, width=10)
latitude.place(relx=0.10, rely=0.1, anchor='center')

# Projected magnitude
Label(window, text="Projected magnitude", background='#dbeff9', font=("Segoe UI", 10, "bold")).place(relx=0.15, rely=0.235, anchor='center')

lbl = Label(window, text="The projected magnitude \n       in your location is:", background='#dbeff9')
lbl.place(relx=0.15, rely=0.27, anchor='center')

lbl = Label(window, text="The category of this magnitude level is:", background='#dbeff9')
lbl.place(relx=0.15, rely=0.40, anchor='center')

# Tips
Label(window, text="Tips", background='#dbeff9', font=("Segoe UI", 10, "bold")).place(relx=0.15, rely=0.535, anchor='center')

# Earthquakes in your area
Label(window, text="Earthquakes in your area (Worldmap View)", background='#dbeff9', font=("Segoe UI", 10, "bold")).place(relx=0.65, rely=0.02, anchor='center')
Label(window, text="Earthquakes in your area (Dataset View)", background='#dbeff9', font=("Segoe UI", 10, "bold")).place(relx=0.65, rely=0.535, anchor='center')
window.mainloop()

C:\Users\Laptop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\collections.py:967: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
C:\Users\Laptop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\collections.py:967: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
C:\Users\Laptop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\matplotlib\collections.py:967: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
